In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),    
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          1

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=80,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 16:12:20.747305: {'loss': 1.767986340522766, 'accuracy': 0.415475, 'val_loss': 1.4432026163028304, 'val_accuracy': 0.4822}


Epoch 1 ends at 16:12:41.907978: {'loss': 1.1952046412467956, 'accuracy': 0.57805, 'val_loss': 1.4438974162575546, 'val_accuracy': 0.5308}


Epoch 2 ends at 16:13:05.268027: {'loss': 1.0252237257003785, 'accuracy': 0.643475, 'val_loss': 0.9370541253666969, 'val_accuracy': 0.6687}


Epoch 3 ends at 16:13:31.095020: {'loss': 0.9356840694427491, 'accuracy': 0.67145, 'val_loss': 1.0256424360214524, 'val_accuracy': 0.6371}


Epoch 4 ends at 16:13:57.651967: {'loss': 0.8749700545310974, 'accuracy': 0.693875, 'val_loss': 0.8725034021268225, 'val_accuracy': 0.6917}


...

Epoch 75 ends at 16:46:48.591433: {'loss': 0.2722012898206711, 'accuracy': 0.900975, 'val_loss': 0.8406642713364522, 'val_accuracy': 0.7639}


Epoch 76 ends at 16:47:17.073059: {'loss': 0.27020555269122126, 'accuracy': 0.90315, 'val_loss': 0.8667297116510427, 'val_accuracy': 0.7549}


Epoch 77 ends at 16:47:45.896003: {'loss': 0.2631460512638092, 'accuracy': 0.9049, 'val_loss': 0.807817159374808, 'val_accuracy': 0.7733}


Epoch 78 ends at 16:48:13.465137: {'loss': 0.2669034168601036, 'accuracy': 0.902525, 'val_loss': 0.7683581047377009, 'val_accuracy': 0.7799}


Epoch 79 ends at 16:48:42.053700: {'loss': 0.2678233012080193, 'accuracy': 0.9017, 'val_loss': 0.8350387917962044, 'val_accuracy': 0.7605}


In [11]:
model.evaluate(test_data)

157/157 [==============================].7654 - accuracy: 0.79 - 0s 52ms/step - loss: 0.7102 - accuracy: 0.78 - 0s 50ms/step - loss: 0.7013 - accuracy: 0.79 - 0s 47ms/step - loss: 0.7346 - accuracy: 0.78 - 0s 49ms/step - loss: 0.6722 - accuracy: 0.79 - 0s 47ms/step - loss: 0.6413 - accuracy: 0.81 - 0s 47ms/step - loss: 0.6629 - accuracy: 0.80 - 0s 46ms/step - loss: 0.7158 - accuracy: 0.78 - 0s 47ms/step - loss: 0.7268 - accuracy: 0.78 - 0s 47ms/step - loss: 0.7670 - accuracy: 0.78 - 0s 45ms/step - loss: 0.7604 - accuracy: 0.79 - 1s 45ms/step - loss: 0.7590 - accuracy: 0.78 - 1s 44ms/step - loss: 0.7588 - accuracy: 0.78 - 1s 44ms/step - loss: 0.7549 - accuracy: 0.78 - 1s 44ms/step - loss: 0.7646 - accuracy: 0.78 - 1s 43ms/step - loss: 0.8016 - accuracy: 0.77 - 1s 43ms/step - loss: 0.8145 - accuracy: 0.77 - 1s 43ms/step - loss: 0.8230 - accuracy: 0.77 - 1s 43ms/step - loss: 0.8227 - accuracy: 0.76 - 1s 42ms/step - loss: 0.8329 - accuracy: 0.76 - 1s 42ms/step - loss: 0.8150 - accuracy: 0.

[0.8590663565192252, 0.7573]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

OSError: SavedModel file does not exist at: checkpoints/20191028-161157.hdf5/{saved_model.pbtxt|saved_model.pb}